In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import sys as _sys
import multitasking as _multitasking


In [2]:
tickers = pd.read_excel("../../data/static_files/nasdaq_nyse.xlsx",header=1)


c:\Users\jliv\Desktop\research\python_research\.venv\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
tickers.dropna(inplace=True)

indexes = ['^GSPC', '^IXIC', '^DJI','^RUT','VOO','VV']

tickers = list(tickers['Symbol'])+indexes

In [4]:
batched_tickers = []

batch_size = 10
batches = int(np.floor(len(tickers)/batch_size))
for count in range(batches+1):
    limit = np.min([len(tickers),batch_size*(count+1)])
    batched_tickers.append(tickers[batch_size*count:limit])
    



In [5]:
batched_tickers[0]

['A',
 'AA',
 'AACG',
 'AAIC',
 'AAIC-B',
 'AAIC-C',
 'AAL',
 'AAM-A',
 'AAM-B',
 'AAME']

In [19]:
tmp = yf.download(  # or pdr.get_data_yahoo(...
            # tickers list or string as well
            tickers = ' '.join(batched_tickers[0]),
            period = "5y",
            interval = "1mo",
            group_by = 'ticker',
            rounding=True,
            auto_adjust = False,
            prepost = False,
            threads = True,
            keepna=False,
            proxy = None,
        )

tmp

[*********************100%***********************]  10 of 10 completed
 Concattenating.. 
 concat ok 
 renaming 
 renaming ok 


{'AAIC-C': Empty DataFrame
 Columns: [Open, High, Low, Close, Adj Close, Volume]
 Index: [],
 'AAM-B': Empty DataFrame
 Columns: [Open, High, Low, Close, Adj Close, Volume]
 Index: [],
 'AAIC-B': Empty DataFrame
 Columns: [Open, High, Low, Close, Adj Close, Volume]
 Index: [],
 'A':                              Open    High     Low   Close  Adj Close  \
 Date                                                                   
 2017-11-01 00:00:00-04:00   68.20   70.93   66.25   69.24      66.63   
 2017-12-01 00:00:00-05:00   69.19   70.00   65.60   66.97      64.44   
 2017-12-29 00:00:00-05:00     NaN     NaN     NaN     NaN        NaN   
 2018-01-01 00:00:00-05:00   67.42   75.00   67.34   73.43      70.81   
 2018-02-01 00:00:00-05:00   73.18   74.99   65.03   68.59      66.15   
 ...                           ...     ...     ...     ...        ...   
 2022-08-01 00:00:00-04:00  133.51  149.00  127.16  128.25     128.03   
 2022-09-01 00:00:00-04:00  127.63  140.00  120.60  121.55  

In [20]:
for k in list(tmp.keys()):

    tmp[k] = tmp[k].to_dict()

tmp

{'AAIC-C': {'Open': {},
  'High': {},
  'Low': {},
  'Close': {},
  'Adj Close': {},
  'Volume': {}},
 'AAM-B': {'Open': {},
  'High': {},
  'Low': {},
  'Close': {},
  'Adj Close': {},
  'Volume': {}},
 'AAIC-B': {'Open': {},
  'High': {},
  'Low': {},
  'Close': {},
  'Adj Close': {},
  'Volume': {}},
 'A': {'Open': {Timestamp('2017-11-01 00:00:00-0400', tz='America/New_York'): 68.2,
   Timestamp('2017-12-01 00:00:00-0500', tz='America/New_York'): 69.19,
   Timestamp('2017-12-29 00:00:00-0500', tz='America/New_York'): nan,
   Timestamp('2018-01-01 00:00:00-0500', tz='America/New_York'): 67.42,
   Timestamp('2018-02-01 00:00:00-0500', tz='America/New_York'): 73.18,
   Timestamp('2018-03-01 00:00:00-0500', tz='America/New_York'): 68.47,
   Timestamp('2018-04-01 00:00:00-0400', tz='America/New_York'): 66.65,
   Timestamp('2018-04-02 00:00:00-0400', tz='America/New_York'): nan,
   Timestamp('2018-05-01 00:00:00-0400', tz='America/New_York'): 65.63,
   Timestamp('2018-06-01 00:00:00-0400'

In [21]:
pd.DataFrame(tmp)

,AAIC-C,AAM-B,AAIC-B,A,AAIC,AA,AAM-A,AAME,AAL,AACG
Open,{},{},{},"{2017-11-01 00:00:00-04:00: 68.2, 2017-12-01 0...","{2017-11-01 00:00:00-04:00: 11.52, 2017-12-01 ...","{2017-11-01 00:00:00-04:00: 48.91, 2017-12-01 ...",{},"{2017-11-01 00:00:00-04:00: 3.7, 2017-12-01 00...","{2017-11-01 00:00:00-04:00: 48.0, 2017-11-10 0...","{2017-11-01 00:00:00-04:00: 4.64, 2017-12-01 0..."
High,{},{},{},"{2017-11-01 00:00:00-04:00: 70.93, 2017-12-01 ...","{2017-11-01 00:00:00-04:00: 11.99, 2017-12-01 ...","{2017-11-01 00:00:00-04:00: 49.27, 2017-12-01 ...",{},"{2017-11-01 00:00:00-04:00: 3.8, 2017-12-01 00...","{2017-11-01 00:00:00-04:00: 51.16, 2017-11-10 ...","{2017-11-01 00:00:00-04:00: 5.3, 2017-12-01 00..."
Low,{},{},{},"{2017-11-01 00:00:00-04:00: 66.25, 2017-12-01 ...","{2017-11-01 00:00:00-04:00: 10.93, 2017-12-01 ...","{2017-11-01 00:00:00-04:00: 40.22, 2017-12-01 ...",{},"{2017-11-01 00:00:00-04:00: 3.08, 2017-12-01 0...","{2017-11-01 00:00:00-04:00: 45.27, 2017-11-10 ...","{2017-11-01 00:00:00-04:00: 4.05, 2017-12-01 0..."
Close,{},{},{},"{2017-11-01 00:00:00-04:00: 69.24, 2017-12-01 ...","{2017-11-01 00:00:00-04:00: 11.81, 2017-12-01 ...","{2017-11-01 00:00:00-04:00: 41.51, 2017-12-01 ...",{},"{2017-11-01 00:00:00-04:00: 3.6, 2017-12-01 00...","{2017-11-01 00:00:00-04:00: 50.49, 2017-11-10 ...","{2017-11-01 00:00:00-04:00: 5.02, 2017-12-01 0..."
Adj Close,{},{},{},"{2017-11-01 00:00:00-04:00: 66.63, 2017-12-01 ...","{2017-11-01 00:00:00-04:00: 8.1167, 2017-12-01...","{2017-11-01 00:00:00-04:00: 41.22, 2017-12-01 ...",{},"{2017-11-01 00:00:00-04:00: 3.5117, 2017-12-01...","{2017-11-01 00:00:00-04:00: 49.1, 2017-11-10 0...","{2017-11-01 00:00:00-04:00: 0.5379, 2017-12-01..."
Volume,{},{},{},"{2017-11-01 00:00:00-04:00: 41983700.0, 2017-1...","{2017-11-01 00:00:00-04:00: 7835000.0, 2017-12...","{2017-11-01 00:00:00-04:00: 86077600.0, 2017-1...",{},"{2017-11-01 00:00:00-04:00: 59600.0, 2017-12-0...","{2017-11-01 00:00:00-04:00: 73187100.0, 2017-1...","{2017-11-01 00:00:00-04:00: 168300.0, 2017-12-..."


In [ ]:
data = pd.DataFrame()
for b in batched_tickers:
    tmp = yf.download(  # or pdr.get_data_yahoo(...
            # tickers list or string as well
            tickers = ' '.join(b),
            period = "5y",
            interval = "1mo",
            group_by = 'column',
            rounding=True,
            auto_adjust = False,
            prepost = False,
            threads = True,
            keepna=False,
            proxy = None,
        )

    data = pd.concat([data,tmp])



In [ ]:
data

In [ ]:



class ProgressBar:
    def __init__(self, iterations, text='completed'):
        self.text = text
        self.iterations = iterations
        self.prog_bar = '[]'
        self.fill_char = '*'
        self.width = 50
        self.__update_amount(0)
        self.elapsed = 1

    def completed(self):
        if self.elapsed > self.iterations:
            self.elapsed = self.iterations
        self.update_iteration(1)
        print('\r' + str(self), end='')
        _sys.stdout.flush()
        print()

    def animate(self, iteration=None):
        if iteration is None:
            self.elapsed += 1
            iteration = self.elapsed
        else:
            self.elapsed += iteration

        print('\r' + str(self), end='')
        _sys.stdout.flush()
        self.update_iteration()

    def update_iteration(self, val=None):
        val = val if val is not None else self.elapsed / float(self.iterations)
        self.__update_amount(val * 100.0)
        self.prog_bar += '  %s of %s %s' % (
            self.elapsed, self.iterations, self.text)

    def __update_amount(self, new_amount):
        percent_done = int(round((new_amount / 100.0) * 100.0))
        all_full = self.width - 2
        num_hashes = int(round((percent_done / 100.0) * all_full))
        self.prog_bar = '[' + self.fill_char * \
            num_hashes + ' ' * (all_full - num_hashes) + ']'
        pct_place = (len(self.prog_bar) // 2) - len(str(percent_done))
        pct_string = '%d%%' % percent_done
        self.prog_bar = self.prog_bar[0:pct_place] + \
            (pct_string + self.prog_bar[pct_place + len(pct_string):])

    def __str__(self):
        return str(self.prog_bar)

@_multitasking.task
def pb_updater(pb):

    pb.animate()

In [ ]:
threads =  _multitasking.cpu_count() * 2
_multitasking.set_max_threads(threads)

for i in range(10):
    
    pb = ProgressBar(500, text='completed')
    for j in range(500):
        pb_updater(pb=pb)
    print("task complete")
